
# 📘 FB3PFB - NHANES project - introduction

This version is **explicit** and **slow** on purpose.  

We will:
1. Import a few libraries and explain each one.
2. Decide where to save data and **download** three NHANES files from the CDC.
3. **Load** the files (SAS XPORT `.xpt`) into pandas DataFrames.
4. **Merge** them by participant ID (`seqn`).
5. **Recode** sex from numbers to text and pick the **SI** lipid columns already provided by NHANES.
6. Produce simple **summaries** and **plots**.
7. Run a **t‑test** and a tiny **regression**, then look at **diagnostics**.
8. Show **next steps** for robust inference and simple model variations.

> Note: This is an unweighted analysis. For official NHANES inference, survey weights / design are required.


## 1) Load libraries

Python is small on purpose. Most useful tools (tables, plotting, stats, downloading files) live in **libraries** - pre-built packages we can reuse instead of writing everything from scratch. In this notebook we import a few focused libraries so we can:

- Read & tidy data (pandas)
- Do basic maths (numpy)
- Download NHANES files (requests)
- Make simple plots (matplotlib)
- Run statistical tests & regressions (scipy, statsmodels)
- Handle file paths safely (pathlib)

Importing them at the start makes the rest of the code shorter, clearer, and less error-prone.


In [ ]:
# The 'sys' module lets us inspect details about Python itself and the environment.
import sys

# 'Path' is a class that represents file/folder paths in a clean, cross-platform way.
# It is nicer and safer than writing strings like 'C:\folder\file.txt'.
from pathlib import Path

# 'pandas' is the main Python library for working with tables of data (DataFrames),
# similar to R's data frames or Excel sheets.
import pandas as pd

# 'numpy' provides fast numeric arrays and useful math utilities.
# Pandas uses numpy internally, and we will use it for a few simple calculations.
import numpy as np

# 'requests' is a small library that lets Python download files from the web (HTTP).
# We'll use it to download the NHANES files directly from the CDC website.
import requests

# 'matplotlib.pyplot' is a plotting library; we will make simple charts (scatter, density, etc.).
import matplotlib.pyplot as plt

# 'scipy.stats' has many statistical tests; we will use it for a Welch's t-test.
from scipy import stats

# 'statsmodels' lets us fit regression models using a friendly formula syntax,
# like in R: y ~ x1 + x2. We will use both the formula API and a few utilities.
import statsmodels.formula.api as smf
import statsmodels.api as sm

# A formal test for heteroskedasticity (changing variance). It is optional,
# but it's good to show how to check assumptions beyond just looking at plots.
from statsmodels.stats.diagnostic import het_breuschpagan

# The prints below tell us which versions are installed. This can help students
# compare their environment to yours when something looks different.
print("Python:", sys.version.split()[0])
print("pandas:", pd.__version__, "| numpy:", np.__version__)
try:
    import scipy
    import statsmodels
    print("scipy:", scipy.__version__, "| statsmodels:", statsmodels.__version__)
except Exception:
    # If the above import fails, it's okay; it's just a version printout.
    pass



## 2) Download the NHANES files from the CDC

**Goal:** Save data to a folder called `data/raw`, whether we run on Colab or locally.  
We also lightly **validate** that the downloaded file looks like an XPT file and not an HTML error page.


In [ ]:
# This variable becomes True if we are running inside Google Colab.
# Colab exposes a special module named 'google.colab'. We just check for its presence.
IN_COLAB = "google.colab" in sys.modules
print("Running in Colab?", IN_COLAB)

def find_repo_root(start: Path) -> Path:
    """Walk upwards from 'start' until we find a '.git' folder (the git repo root).
    If none is found within ~10 levels, return 'start'.
    This is a convenience for local use: if you are inside a git repository,
    we want to put 'data/' at the repository root. Otherwise, we just use the current folder.
    """
    # .resolve() returns an absolute, clean path (expands any '..' portion).
    p = start.resolve()

    # Try up to 10 levels up the directory tree.
    for _ in range(10):
        # The presence of a '.git' folder normally marks the root of a git repository.
        if (p / ".git").exists():
            return p

        # If p.parent == p, it means we reached the filesystem root (no more parents).
        if p.parent == p:
            break

        # Move one directory up and try again.
        p = p.parent

    # If we didn't find a git root, return the starting point (absolute path).
    return start.resolve()

# Decide where our ROOT folder is:
# - On Colab, '/content' is the standard writable workspace.
# - Locally (e.g., Phoenix/Jupyter), try to locate the git repository root.
#   If not inside a repo, we fall back to the current working directory.
if IN_COLAB:
    ROOT = Path("/content")
else:
    ROOT = find_repo_root(Path.cwd())

print("ROOT folder:", ROOT)

# Create 'data/raw' for downloaded files and 'data/processed' for anything we might save later.
DATA_DIR = ROOT / "data" / "raw"
PROC_DIR = ROOT / "data" / "processed"

# The 'mkdir' calls below create these folders if they don't exist.
# 'parents=True' means: also create the parent folders that might be missing.
# 'exist_ok=True' means: do NOT crash if the folder already exists.
DATA_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_DIR:", DATA_DIR)
print("PROC_DIR:", PROC_DIR)

# These are the CDC URLs for the 2017–2018 NHANES cycle (the files end with '_J').
# If a URL changes in the future, update it here.
FILES = {
    "DEMO_J.xpt":   "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DEMO_J.xpt",
    "HDL_J.xpt":    "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/HDL_J.XPT",
    "TRIGLY_J.xpt": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/TRIGLY_J.XPT",
}

def looks_like_html(blob: bytes) -> bool:
    """Return True if the first bytes look like an HTML page (e.g., an error page).
    We check only a small portion (the first 200 bytes)."""
    head = blob[:200].lower()
    return b"<html" in head or b"<!doctype html" in head

def looks_like_xpt(blob: bytes) -> bool:
    """Return True if the file looks like a SAS XPORT (XPT) file.
    NHANES XPT files typically start with: b"HEADER RECORD*******LIBRARY"""
    return blob.startswith(b"HEADER RECORD*******LIBRARY")

# Some servers can behave differently depending on the "User-Agent".
# We mimic a common browser user agent string to be safe.
headers = {"User-Agent": "Mozilla/5.0"}

# Loop through the files we want to download.
for fname, url in FILES.items():
    # Make a full path like '.../data/raw/DEMO_J.xpt'
    dest = DATA_DIR / fname

    # If the file already exists AND looks like a real XPT, we keep it (no re-download).
    if dest.exists():
        content = dest.read_bytes()

        # We check size and also whether it looks like HTML.
        if len(content) > 500 and not looks_like_html(content):
            print("OK: already have", fname, f"({len(content):,} bytes)")
            continue
        else:
            print("Existing file looks suspicious; removing:", fname)
            try:
                dest.unlink()
            except FileNotFoundError:
                # If the file vanished between 'exists()' and 'unlink()', ignore it.
                pass

    # If we reach here, we need to download the file.
    print("Downloading:", fname)
    response = requests.get(url, headers=headers, timeout=60)
    response.raise_for_status()
    blob = response.content

    # If the server returned an HTML page (very likely an error), stop with a helpful message.
    if looks_like_html(blob):
        print("The server returned HTML for:", fname)
        print("First bytes:", blob[:120])
        raise RuntimeError("Download returned HTML. Try Colab or check your network.")

    # Some XPTs may not have the classic header, but NHANES usually does.
    # We warn the user, but still save the file so we can inspect it.
    if not looks_like_xpt(blob):
        print("Warning: XPT header not detected for", fname)

    # Actually save the downloaded bytes to disk.
    dest.write_bytes(blob)
    print("Saved:", fname, f"({len(blob):,} bytes)")

# Show a short listing of the files we now have in DATA_DIR.
print("\nFiles in DATA_DIR:")
for p in sorted(DATA_DIR.iterdir()):
    size = p.stat().st_size
    print(" -", p.name, f"({size:,} bytes)")



## 3) Load the downloaded files into pandas DataFrames

- We read `.xpt` files using `pandas.read_sas(..., format="xport")`.  
- We then **lower‑case all column names** so we don't have to remember exact capitalization.


In [ ]:
# Build 'Path' objects that point to each file we downloaded.
p_demo = DATA_DIR / "DEMO_J.xpt"
p_hdl = DATA_DIR / "HDL_J.xpt"
p_trigly = DATA_DIR / "TRIGLY_J.xpt"

# Read each SAS XPORT file into a pandas DataFrame.
# The 'encoding' helps with non-ASCII labels in the metadata.
df_demo = pd.read_sas(p_demo, format="xport", encoding="utf-8")
df_hdl = pd.read_sas(p_hdl, format="xport", encoding="utf-8")
df_trig = pd.read_sas(p_trigly, format="xport", encoding="utf-8")

# Convert all column names to lowercase to avoid case sensitivity issues later.
df_demo.columns = [c.lower() for c in df_demo.columns]
df_hdl.columns = [c.lower() for c in df_hdl.columns]
df_trig.columns = [c.lower() for c in df_trig.columns]

# Show the shapes (rows, columns) of the three DataFrames so we know what we loaded.
print("Shapes (rows, cols): DEMO, HDL, TRIGLY")
print(df_demo.shape, df_hdl.shape, df_trig.shape)

# Show the first 3 rows of the DEMO dataset for a quick peek.
df_demo.head(3)


## 4) Merge the three tables by `seqn` (participant ID)

- `seqn` uniquely identifies participants in NHANES and is used to join tables.
- We keep only a few DEMO columns to stay tidy, then **left‑join** the lab tables.


In [ ]:
# From DEMO, keep only the columns we need for this lesson:
# - 'seqn'      = participant ID
# - 'riagendr'  = sex (1 = Male, 2 = Female)
# - 'ridageyr'  = age in years
demo_keep = ["seqn", "riagendr", "ridageyr"]
d = df_demo[demo_keep].copy()

# Merge HDL and TRIGLY by 'seqn'.
# 'how="left"' means: keep everyone from DEMO, even if HDL/TRIGLY is missing for them.
# 'drop_duplicates(subset=["seqn"])' is defensive (ensures we don't duplicate participants if rare issues exist).
d = d.merge(df_hdl.drop_duplicates(subset=["seqn"]), on="seqn", how="left")
d = d.merge(df_trig.drop_duplicates(subset=["seqn"]), on="seqn", how="left")

# .info() prints a summary of columns and data types.
d.info()


## 5) Recode sex and pick the **SI** lipid columns

- NHANES encodes sex in `riagendr` as **1 → Male** and **2 → Female**. We create a new `sex` column with text labels.
- NHANES also provides **SI units** for some lipids (already in **mmol/L**):
  - HDL: `lbdhddsi`  
  - LDL: `lbdldlsi`  
  - Triglycerides: `lbdtrsi`  
We create new, friendly column names **only if** these SI columns exist.


In [ ]:
# Make sure 'riagendr' is numeric so the mapping works even if the column came in as strings.
d["riagendr"] = pd.to_numeric(d["riagendr"], errors="coerce")

# Create a new 'sex' column with human-friendly labels.
d["sex"] = d["riagendr"].map({1: "Male", 2: "Female"})

# The SI columns we want to use if they exist.
hdl_si = "lbdhddsi"
ldl_si = "lbdldlsi"
tg_si = "lbdtrsi"

# We will keep track of which columns we managed to create.
created = []

# If the HDL SI column exists, create a friendly alias 'hdl_mmol_l' and coerce to numeric.
if hdl_si in d.columns:
    d["hdl_mmol_l"] = pd.to_numeric(d[hdl_si], errors="coerce")
    created.append("HDL")
else:
    print("Note: HDL SI column 'lbdhddsi' not found.")

# Same approach for LDL.
if ldl_si in d.columns:
    d["ldl_mmol_l"] = pd.to_numeric(d[ldl_si], errors="coerce")
    created.append("LDL")
else:
    print("Note: LDL SI column 'lbdldlsi' not found.")

# Same approach for triglycerides.
if tg_si in d.columns:
    d["tg_mmol_l"] = pd.to_numeric(d[tg_si], errors="coerce")
    created.append("TG")
else:
    print("Note: TG SI column 'lbdtrsi' not found.")

print("Created columns:", created)

# Build a list of key columns to show. We only include lipid columns if they exist.
cols_to_show = ["sex", "ridageyr"]
if "hdl_mmol_l" in d.columns:
    cols_to_show.append("hdl_mmol_l")
if "ldl_mmol_l" in d.columns:
    cols_to_show.append("ldl_mmol_l")
if "tg_mmol_l" in d.columns:
    cols_to_show.append("tg_mmol_l")

# Display the first few rows of these columns to confirm everything looks right.
d[cols_to_show].head()


## 6) Describe age by sex (n, mean ± SD, median [IQR])

We will:
1. Select ages within each sex.
2. Drop missing values (so stats are computed on valid numbers only).
3. Compute and print simple summaries.


In [ ]:
# A simple loop over the two sex groups we expect.
print("=== Age summary by sex ===")

for grp in ["Male", "Female"]:
    # Select only the 'ridageyr' values (age) for the chosen sex and remove missing values (NaN).
    sub = d.loc[d["sex"] == grp, "ridageyr"].dropna()

    # If there are no rows for this group, print a message and skip to the next group.
    if sub.empty:
        print()
        print(grp + ": no data")
        continue

    # Compute summary statistics on the 1D array (a pandas Series).
    n = sub.size                             # how many ages do we have?
    mean = sub.mean()                        # average age
    sd = sub.std(ddof=1)                     # sample standard deviation (divide by n-1)
    median = sub.median()                    # middle value
    q25 = sub.quantile(0.25)                 # 25th percentile (first quartile)
    q75 = sub.quantile(0.75)                 # 75th percentile (third quartile)

    # Print the results in a readable way.
    print()
    print(grp + ": n=" + str(n))
    print("  Mean ± SD:    {:.1f} ± {:.1f}".format(mean, sd))
    print("  Median [IQR]: {:.1f} [{:.1f}, {:.1f}]".format(median, q25, q75))


## 7) Density plot of HDL (mmol/L) by sex

We draw a **KDE** (Kernel Density Estimate) curve for HDL for each sex.  
The x‑axis is forced to start at **0** (HDL cannot be negative).


In [ ]:
# Prepare the two 1D arrays (Series), one for each sex.
# We remove missing values because plotting functions expect numbers.
hdl_f = d.loc[d["sex"] == "Female", "hdl_mmol_l"].dropna()
hdl_m = d.loc[d["sex"] == "Male", "hdl_mmol_l"].dropna()

# A small safety check: report if any negative HDL values exist.
neg_f = (hdl_f < 0).sum()
neg_m = (hdl_m < 0).sum()
if neg_f > 0 or neg_m > 0:
    print("Warning: negative HDL values detected — Female:", neg_f, " Male:", neg_m)

# Create a new figure (plot).
plt.figure()

# Ask pandas to plot a KDE (smooth density curve) for each group.
# 'linewidth=2' makes the line thicker and easier to see.
# 'label' gives a name used in the legend.
hdl_f.plot(kind="kde", linewidth=2, label="Female")
hdl_m.plot(kind="kde", linewidth=2, label="Male")

# Add labels and a title so the chart is self-explanatory.
plt.xlabel("HDL (mmol/L)")
plt.ylabel("Density")
plt.title("HDL distribution by sex")

# Show the legend (mapping colors to labels) and a faint grid.
plt.legend()
plt.grid(alpha=0.2)

# Force the x-axis to start at 0 (HDL cannot be negative).
plt.xlim(left=0)

# Finally, draw the plot.
plt.show()


## 8) Mean difference and Welch’s t‑test (Female − Male)

- Compute unweighted mean HDL for women and for men.  
- Calculate the difference (Female − Male).  
- Use **Welch’s t‑test** (`equal_var=False`) which is more reliable if the two groups have different variances.


In [ ]:
# Compute the mean HDL for each group (these are simple averages).
hdl_mean_f = hdl_f.mean()
hdl_mean_m = hdl_m.mean()

# Subtract to get the difference.
difference = hdl_mean_f - hdl_mean_m

# Print the means and the difference nicely formatted.
print("Mean HDL (mmol/L): Female = {:.2f}, Male = {:.2f}".format(hdl_mean_f, hdl_mean_m))
print("Difference (Female − Male): {:.2f} mmol/L".format(difference))

# Welch's t-test compares the means of two groups without assuming equal variances.
# 'nan_policy="omit"' tells the function to ignore any missing values (though we already dropped them).
t_stat, p_val = stats.ttest_ind(hdl_f, hdl_m, equal_var=False, nan_policy="omit")
print("Welch's t-test: Female vs Male")
print("  t = {:.3f}, p = {:.3g}".format(t_stat, p_val))
print("  n_female =", hdl_f.size, " n_male =", hdl_m.size)


## 9) Scatter: HDL (mmol/L) vs Age (years), colored by sex

This is a raw visualization; it does not imply a linear relationship — it just shows the cloud of points.


In [ ]:
# Build a temporary filtered view that has no missing age or HDL.
clean = d.dropna(subset=["ridageyr", "hdl_mmol_l"])

# Create a new figure for the scatter plot.
plt.figure()

# Group rows by 'sex' so that each group can be plotted with its own color/label.
# 'dropna=False' keeps rows where 'sex' might be missing (they will be labeled 'nan').
for label, sub in clean.groupby("sex", dropna=False):
    plt.scatter(sub["ridageyr"], sub["hdl_mmol_l"], s=10, alpha=0.35, label=str(label))

# Label axes and add a title.
plt.xlabel("Age (years)")
plt.ylabel("HDL (mmol/L)")
plt.title("HDL vs Age by sex")

# Constrain axes to sensible minimums (HDL and age should not be negative).
plt.xlim(left=0)
plt.ylim(bottom=0)

# Show the legend and a light grid.
plt.legend()
plt.grid(alpha=0.2)

# Draw the scatter plot.
plt.show()


## 10) Pre‑regression checks

We quickly check:
- **Missing values** for our model variables,
- A **boxplot** of HDL by sex,
- A raw **scatter** of HDL vs age again (all participants),
- A simple **correlation** between HDL and age.


In [ ]:
# A) Count missing values for the variables we plan to use in the model.
print("Missing counts:")
print(d[["hdl_mmol_l", "sex", "ridageyr"]].isna().sum())

# B) Boxplot of HDL by sex. This shows medians and spread/group differences.
plt.figure()
d.boxplot(column="hdl_mmol_l", by="sex")
plt.ylabel("HDL (mmol/L)")
plt.title("HDL by sex")
plt.suptitle("")  # removes the automatic subtitle 'Boxplot grouped by sex'
plt.show()

# C) Raw scatter again (everyone together), for context.
plt.figure()
plt.scatter(d["ridageyr"], d["hdl_mmol_l"], s=10, alpha=0.35)
plt.xlabel("Age (years)")
plt.ylabel("HDL (mmol/L)")
plt.title("HDL vs Age (raw)")
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.grid(alpha=0.2)
plt.show()

# D) Correlation between HDL and age (numeric-only).
# This is just a single number summarizing linear association; it does not imply causation.
print()
print("Correlation (numeric cols):")
print(d[["hdl_mmol_l", "ridageyr"]].corr(numeric_only=True))


## 11) OLS regression: `HDL (mmol/L) ~ sex + age`

- We build a clean dataset with the variables we need and drop rows with missing values.  
- We treat `sex` as **categorical** (with `C(sex)`), so the model estimates the difference between Male and the baseline (Female).  
- We print a full summary that includes coefficients, standard errors, p-values, and R².


In [ ]:
# Build a new DataFrame for the model with just the needed columns.
reg = d[["hdl_mmol_l", "sex", "ridageyr"]].dropna().copy()

# Convert the numeric columns to numbers explicitly (safe practice),
# even though they should already be numeric from earlier steps.
reg["hdl_mmol_l"] = pd.to_numeric(reg["hdl_mmol_l"], errors="coerce")
reg["ridageyr"] = pd.to_numeric(reg["ridageyr"], errors="coerce")

# Drop any row that turned into NaN after coercion.
reg = reg.dropna(subset=["hdl_mmol_l", "ridageyr"])

# Fit the linear model using the R-like formula interface.
# 'C(sex)' tells statsmodels to treat 'sex' as a categorical variable (i.e., create dummy variables).
model = smf.ols("hdl_mmol_l ~ C(sex) + ridageyr", data=reg).fit()

# Print a detailed summary table.
print(model.summary())


## 12) Diagnostics: residuals and formal tests

We check:
- **Residuals vs fitted** (linearity + constant variance),
- **Q–Q plot** of standardized residuals (normality),
- **Scale–location** plot (variance vs fitted),
- **Breusch–Pagan** test for heteroskedasticity,
- **D’Agostino’s K²** test for normality of residuals.


In [ ]:
# Get fitted values (ŷ) and residuals (observed y minus fitted ŷ).
fitted = model.fittedvalues
resid = model.resid

# Compute standardized (studentized internal) residuals for scale-free plots.
influence = model.get_influence()
std_resid = influence.resid_studentized_internal

# 1) Residuals vs Fitted: should look like random noise around 0 if the model is ok.
plt.figure()
plt.scatter(fitted, resid, s=12, alpha=0.5)
plt.axhline(0, color="black", linewidth=1)
plt.xlabel("Fitted values (ŷ)")
plt.ylabel("Residuals (e = y − ŷ)")
plt.title("Residuals vs Fitted")
plt.grid(alpha=0.2)
plt.show()

# 2) Q–Q plot: if residuals are normal, points will be near the 45-degree line.
plt.figure()
sm.qqplot(std_resid, line="45", fit=True)
plt.title("Q–Q plot (standardized residuals)")
plt.grid(alpha=0.2)
plt.show()

# 3) Scale–Location: if variance is constant, this should be a roughly flat band.
plt.figure()
plt.scatter(fitted, np.sqrt(np.abs(std_resid)), s=12, alpha=0.5)
plt.xlabel("Fitted values (ŷ)")
plt.ylabel("sqrt(|standardized residuals|)")
plt.title("Scale–Location (variance check)")
plt.grid(alpha=0.2)
plt.show()

# 4) Breusch–Pagan test: null hypothesis is constant variance (homoskedasticity).
bp_lm, bp_pvalue, f_stat, f_pvalue = het_breuschpagan(resid, model.model.exog)
print("Breusch–Pagan test (heteroskedasticity):")
print("  LM stat = {:.3f},  p = {:.3g}   (F p = {:.3g})".format(bp_lm, bp_pvalue, f_pvalue))

# 5) D’Agostino’s K² normality test: null hypothesis is normal residuals.
k2_stat, k2_p = stats.normaltest(std_resid, nan_policy="omit")
print("D’Agostino’s K² test (normality of residuals):")
print("  K² = {:.3f},  p = {:.3g}".format(k2_stat, k2_p))


### Model diagnostics — summary
- **Model fit:** `HDL (mmol/L) ~ sex + age`. Two horizontal bands of fitted values suggest the **sex effect** is large; the **age slope** looks small.  
- **Residual normality:** Q–Q plot often shows a **heavy right tail** → some high positive residuals.  
- **Homoskedasticity:** Residuals vs fitted and scale–location can show **funnel shapes** → likely **heteroskedasticity**.  
- **Implication:** OLS coefficients are still average effects, but **standard errors/p‑values** from plain OLS can be unreliable.

**Recommended next steps** (for more rigorous inference):
- Report **robust standard errors (HC3)** for your regression.
- Consider mild **nonlinearity in age** (quadratic or spline) if residuals show curvature.
- Do sensitivity checks (e.g., **winsorize** extreme HDL or use a **robust regression**).



## 13) Next steps (optional, but good practice)

We now try a few reasonable follow‑ups:
- **Stratified models:** fit `HDL ~ age` separately for women and men.
- **Pooled model with HC3 robust SEs:** `HDL ~ sex + age` with more reliable SEs.
- **Quadratic age:** `HDL ~ sex + age + age²` if slight curvature exists.
- **Winsorize** (cap top 1% of HDL) as a robustness check.
- **Interaction:** test whether the **age slope differs by sex**.


In [ ]:
# Rebuild the modeling dataset cleanly (so a student can re-run this cell alone).
reg = d[["hdl_mmol_l", "sex", "ridageyr"]].dropna().copy()
reg["hdl_mmol_l"] = pd.to_numeric(reg["hdl_mmol_l"], errors="coerce")
reg["ridageyr"] = pd.to_numeric(reg["ridageyr"], errors="coerce")
reg = reg.dropna(subset=["hdl_mmol_l", "ridageyr"])

print("Rows available for modeling:", len(reg))
print(reg[["sex"]].value_counts().rename_axis("sex").to_frame("n"))


### 13a) Stratified models (by sex) with HC3 robust SEs


In [ ]:
def fit_stratified(df, sex_label):
    """Fit OLS: HDL ~ age for one sex, using HC3 robust standard errors.
    Prints a small coefficient table and the age slope with its p-value.
    """
    subset = df[df["sex"] == sex_label]

    if subset.empty:
        print("[" + sex_label + "] No data — skipping.")
        return None

    model_strat = smf.ols("hdl_mmol_l ~ ridageyr", data=subset).fit(cov_type="HC3")
    print()
    print("[" + sex_label + "] OLS: HDL ~ age (HC3 SEs)")
    print(model_strat.summary().tables[1])

    slope = model_strat.params.get("ridageyr", np.nan)
    pval = model_strat.pvalues.get("ridageyr", np.nan)
    n_obs = int(model_strat.nobs)

    print("  Slope (per year): {:.4f} mmol/L  (p={:.3g})  |  n={}".format(slope, pval, n_obs))

    return model_strat

mod_women = fit_stratified(reg, "Female")
mod_men = fit_stratified(reg, "Male")


### 13b) Pooled model with HC3 robust SEs


In [ ]:
model_hc3 = smf.ols("hdl_mmol_l ~ C(sex) + ridageyr", data=reg).fit(cov_type="HC3")
print("[Pooled] OLS: HDL ~ sex + age (HC3 SEs)")
print(model_hc3.summary().tables[1])


### 13c) Add a mild nonlinearity: quadratic age


In [ ]:
model_quad = smf.ols("hdl_mmol_l ~ C(sex) + ridageyr + I(ridageyr**2)", data=reg).fit(cov_type="HC3")
print("[Pooled] Quadratic age (HC3): HDL ~ sex + age + age^2")
print(model_quad.summary().tables[1])


### 13d) Optional: winsorize the top 1% of HDL

This reduces the influence of extreme values. It is a **sensitivity analysis** (report it transparently).


In [ ]:
do_winsorize = True  # set to False if you do not want to run this

if do_winsorize:
    # Compute the 99th percentile and cap values above this threshold.
    cap = reg["hdl_mmol_l"].quantile(0.99)

    reg_w = reg.copy()
    reg_w.loc[reg_w["hdl_mmol_l"] > cap, "hdl_mmol_l"] = cap

    model_wins = smf.ols("hdl_mmol_l ~ C(sex) + ridageyr", data=reg_w).fit(cov_type="HC3")
    print("[Pooled] Winsorized at 99th pct (cap={:.3f}): HDL ~ sex + age (HC3)".format(cap))
    print(model_wins.summary().tables[1])

    # Compare key coefficients side by side (teaching aid).
    def pick(m):
        s = pd.Series({
            "Intercept": m.params.get("Intercept", np.nan),
            "C(sex)[T.Male]": m.params.get("C(sex)[T.Male]", np.nan),
            "ridageyr": m.params.get("ridageyr", np.nan)
        })
        return s

    comp = pd.concat({
        "HC3 pooled": pick(model_hc3),
        "Winsorized (99%)": pick(model_wins),
        "Quadratic age": pd.Series({
            "Intercept": model_quad.params.get("Intercept", np.nan),
            "C(sex)[T.Male]": model_quad.params.get("C(sex)[T.Male]", np.nan),
            "ridageyr": model_quad.params.get("ridageyr", np.nan)
        })
    }, axis=1)

    print()
    print("Key coefficients (comparison):")
    display(comp.round(4))


### 13e) Optional: interaction model

`HDL ~ C(sex) * age` tests whether the **age slope differs by sex** (interaction term).


In [ ]:
mod_int = smf.ols("hdl_mmol_l ~ C(sex) * ridageyr", data=reg).fit(cov_type="HC3")
print("[Pooled] Interaction: HDL ~ C(sex) * age (HC3)")
print(mod_int.summary().tables[1])
print()
print("The coefficient C(sex)[T.Male]:ridageyr tests whether the age slope differs by sex.")